In [11]:
import pandas as pd
import numpy as np
from os import path
from CSVUtils import *
import ta
import matplotlib.pyplot as plt
import seaborn as sn
import calendar
from pprint import pprint

In [12]:
DIR = "./from github/Stock-Trading-Environment/data"
nameList = ["^BVSP", "^TWII", "^IXIC"]
df_list = []
startDate = pd.to_datetime("2015-01-01")
endDate = pd.to_datetime("2019-12-31")

for name in nameList:
    df = csv2df(csv_path=DIR, csv_name=name+".csv", source="yahoo")
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.dropna()
    df_list.append(df)

In [13]:
twd_df = pd.read_csv("./input/currency/TWD_1995-2020.csv")
brl_df = pd.read_csv("./input/currency/BRL_1995-2020.csv")

twd_df['Date'] = pd.to_datetime(twd_df['Date'])
brl_df['Date'] = pd.to_datetime(brl_df['Date'])

In [14]:
twd_change = twd_df[['Date','Change']]
brl_change = brl_df[['Date','Change']]

In [15]:
def stem_df(df, intersect_dates, startDate, endDate):
    df.dropna(inplace = True)
    df = df[df['Date'].isin(intersect_dates)].reset_index(drop=True)
    df = df[(df['Date'] >= startDate) & (df['Date'] <= endDate)]
    df = df.reset_index(drop=True)
    return df

In [16]:
intersect_dates = df_list[0]['Date']
for df in df_list[1:]+[twd_change,brl_change]:
    df.dropna(inplace = True)
    intersect_dates = np.intersect1d(intersect_dates, df['Date'])

/Users/lzcai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
for i, df in enumerate(df_list):
    df_list[i] = stem_df(df, intersect_dates, startDate - pd.Timedelta(days=365), endDate)
twd_change = stem_df(twd_change, intersect_dates, startDate - pd.Timedelta(days=365), endDate)
brl_change = stem_df(brl_change, intersect_dates, startDate - pd.Timedelta(days=365), endDate)

/Users/lzcai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df_list[0]['FX Change'] = brl_change['Change']
df_list[1]['FX Change'] = twd_change['Change']
df_list[2]['FX Change'] = 0

In [19]:
for i, df in enumerate(df_list):
    df['Cum FX Change'] = (df['FX Change']+1).cumprod()
    df['Actual Price'] = df['Price']*df['Cum FX Change']

In [22]:
df_list[2].head()

,Date,Open,High,Low,Price,Vol,Change,FX Change,Cum FX Change,Actual Price
0,2014-01-02,4160.029785,4160.959961,4131.790039,4143.069824,1738820000,-0.008026,0,1,4143.069824
1,2014-01-03,4148.560059,4152.959961,4124.959961,4131.910156,1667480000,-0.002694,0,1,4131.910156
2,2014-01-06,4137.029785,4139.779785,4103.750000,4113.680176,2292840000,-0.004412,0,1,4113.680176
3,2014-01-07,4128.569824,4158.180176,4126.479980,4153.180176,2278220000,0.009602,0,1,4153.180176
4,2014-01-08,4154.279785,4171.750000,4145.000000,4165.609863,2345220000,0.002993,0,1,4165.609863


In [23]:
for i, df in enumerate(df_list):
    df.to_csv(path.join(DIR, nameList[i]+"_new.csv"), index=False)